Version 0.0.2

Voraussetzung:
 - Grundlagen von Python

Enthält:
 - notwendige Implementierungen für das Widget.


Alle im nachfolgenden Implementierten Funktionen/Rechnungen können für die Aufgaben entfernt und durch das Kommentar '# May be implemented by pupils' gekennzeichnet werden, um von den Schülern ergänzt zu werden. Es können und sollen zusätzliche Aufgaben hinzugefügt oder abgeändert werden. Ausschlaggebend hierfür ist das Vorwissen der Schüler und die Schwerpunktsetzung der Themen.

In [ ]:
import sys
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
sys.path.append("..")

Grundidee ist die Simulation mithilfe von Computern. Das zweit einfachste Beispiel ist mit einem Würfel realisiert, das im Idealfall ein Laplace-Experiment darstellt - die auftretenden Ereignisse einer endlichen Menge an möglichen Ereignissen sind gleichberechtigt.

In [ ]:
a = np.random.choice(a=[1,2,3,4,5,6], size=30)
print(a)

Es bietet sich auch die Möglichkeit die auftretenden Ergebnisse unterschiedlich zu gewichten, also einen nicht idealen Würfel zu simulieren. Die Gesamtwahrscheinlichkeit sollte hierbei jedoch immer 1 bleiben.

In [ ]:
b = np.random.choice(a=[1,2,3,4,5,6], p=[0.125/2., 0.25, 0.5, 0.125/4., 0.125/4, 0.125], size=30)
print(b)

Doch bleiben wir bei dem idealen Würfel. Für die spätere Anwendung werden einzelne Funktionen gebraucht. Es lässt sich eine einfache Funktion definieren, die beim Aufruf eine zufällige Zahl aus der Menge $\{1,2,3,4,5,6\}$ ausgibt.

In [ ]:
# May be implemented by pupils

def my_random_dice_roll_simulation(n_times):
    return np.random.choice(a=[1,2,3,4,5,6], p=[1/6, 1/6, 1/6, 1/6, 1/6, 1/6], size=n_times)

for _ in range(5):
    print(my_random_dice_roll_simulation(1)[0])
print(my_random_dice_roll_simulation(5))

Eine Liste aus 30 oder mehr Elementen ist unübersichtlich. Es ist einfacher nur zu zählen wie oft eine Zahl gewürfelt wurde, da jeder Wurf unabhängig vom anderen geschieht. Eine eigenständige explizite Implementierung lässt sich in diesem Fall mit drei Zeilen bewerkstelligen.

In [ ]:
# May be implemented by pupils

# Wurfergebnis     1, 2, 3, 4, 5, 6
a_hist = np.array([0, 0, 0, 0, 0, 0])
for num in a:
    a_hist[num -1] += 1
print(a)
print(a_hist)

Der nullte Eintrag entspricht dem Wurf einer Eins, der erste einer Zwei usw.

Dieser und die darauf aufbauenden Schritte können in einer grafische Anwendung eingebaut werden und schafft eine interaktive Visualisierung der bisher implementierten Größen und Funktionen:

In [ ]:
from include.widget.WuerfelWidgetMyBinderVersion import WuerfelWidget as WW

ww = WW(language="DE")
ww.own_simulation_func = my_random_dice_roll_simulation
ww.run

Nach der Durchführung eines solchen Experimentes ist es sinnvoll einige Größen zu berechnen, um die Messung in irgendeiner weise zu quantifizieren. Als erstens bietet sich der Mittelwert an, welcher direkt in einer Funktion berechnet werden kann:

In [ ]:
# May be implemented by pupils

def mittelwert(my_array):
    my_array = np.array(my_array)
    # return np.mean(my_array)
    return np.sum(my_array) / len(my_array)

Es stellt sich jedoch ein Problem heraus: Der so berechnete Mittelwert des Histogramms ist nur der Mittelwert der Anzahl der Würfe - repräsentiert also die Erwartung wie oft beispielsweise, die fünf gewürfelt wird. Diese Größe wird aber auch im weiteren eine Anwendung finden.

In [ ]:
print(mittelwert(a))
print(mittelwert(a_hist))

Die Berechnung des eigentlichen Mittelwertes für ein Histogramm lässt sich mit der folgenden Formel realisieren: $$ \bar{x} = \frac{\sum_{i=1}^6 i \cdot n_i}{\sum_{i=1}^6 n_i}  $$ wobei $i$ der jeweilige Bin ist und $n_i$ die Anzahl an Ereignissen für dieses Bin ist.

Als Erstes ist es empfehlenswert die Normierung in einer separaten Funktion darzustellen, da diese für alle nachfolgenden Rechnungen gleich sein wird:

In [ ]:
# May be implemented by pupils

def normierung(my_array):
    my_array = np.array(my_array)
    summe = np.sum(my_array)
    if summe == 0.0:
        return np.array(my_array)
    return (1./summe) * np.array(my_array)

Die nun so normierten Histogrammeinträge sind:

In [ ]:
a_hist_normiert = normierung(a_hist)
print(a_hist)
print(a_hist_normiert)

Und entsprechen den Wahrscheinlichkeiten die jeweilige Zahl zu werfen.
Mithilfe der normierten Histogrammeinträge lässt sich im nächsten Schritt der Mittelwert berechnen:

In [ ]:
# May be implemented by pupils

def mittelwert_histogramm(my_array):
    my_array = np.array(my_array) if np.sum(my_array) == 1.0 else normierung(my_array)
    # return sum(i* item for i, item in enumerate(my_array, start=1))
    my_bins = np.array([i for i in range(1, len(my_array) + 1)])
    return np.sum(my_bins * my_array)

In [ ]:
print(mittelwert(a))
print(mittelwert_histogramm(a_hist))
print(mittelwert_histogramm(a_hist_normiert))

In [ ]:
ww.own_mean_func = mittelwert
ww.own_norm_func = normierung
ww.run

Die nächste Größe ist die Standardabweichung der Gesamtmessung: $$\sigma = \sqrt{\frac{1}{6 - k} \sum_{i=1}^6 (x_i - \bar{x})^2}\, ,$$wobei hier wieder die Standardabweichung der Gesamtmessung für das Histogramm anders berechnen lässt: $$\sigma = \sqrt{\frac{\sum_{i=1}^6 i^2 n_i}{-k + \sum_{i=1}^6 n_i}}$$

In [ ]:
# May be implemented by pupils

def standardabweichung(my_array, k=0.0):
    mittelwert_ = mittelwert(my_array)
    return np.sqrt((1./(len(my_array - k))) * np.sum((my_array - mittelwert_) ** 2))

def standardabweichung_histogramm(my_array):
    my_array = np.array(my_array) if np.sum(my_array) == 1.0 else normierung(my_array)
    mittelwert_ = mittelwert_histogramm(my_array)
    # return np.sqrt(sum((i - mittelwert_) ** 2 * item for i, item in enumerate(my_array, start=1)))
    my_bins = np.array([i for i in range(1, len(my_array) + 1)])
    varianz = np.sum((my_bins - mittelwert_) ** 2 * my_array)
    return np.sqrt(varianz)

In [ ]:
print(standardabweichung(a))
print(standardabweichung_histogramm(a_hist_normiert))

In [ ]:
ww.own_std_all_func = standardabweichung
ww.run

Ebenfalls ist es interessant die Frage welche Unsicherheit für fünfmal den Wurf fünf gab, zu beantworten. Hierzu kann jeder Bineintrag als ein eigenständiger Poisson Prozess betrachtet werden. Die Unsicherheit ist im einfachsten Fall gegeben als die Wurzel der Ereignisse.

In [ ]:
# May be implemented by pupilsa

def unsicherheit_poisson(array):
    return np.sqrt(array)

In [ ]:
print(unsicherheit_poisson(2))
print(a_hist)
print(unsicherheit_poisson(a_hist))

In [ ]:
ww.own_std_indv_func = unsicherheit_poisson
ww.run

Für die grafische Darstellung werden nun alle Funktionen zusammengefasst. Es ist noch sinnvoll eine Skalierung der simulierten Ereignisse an die Menge der durchgeführten Messungen durchzuführen. Dadurch wird eine Vergleichbarkeit der Erwartung mit der tatsächlichen Messung erreicht. Für eine gute Vorhersage ist es sinnvoll eine größere Menge an Ereignissen zu simulieren, um dann mit der eigentlichen Messung zu vergleichen.

In [ ]:
# May be implemented by pupils

def skalierung_der_simulation_an_die_messung(messung):
    return 1.0 if np.sum(messung) == 0.0 else np.sum(messung)

In [ ]:
ww.own_measurement_scale_func = skalierung_der_simulation_an_die_messung
ww.own_measurement_func = my_random_dice_roll_simulation
ww.run

Zusatz:  
Am Schluss soll eine Größe Implementiert werden das Bewerten soll, ob die vorhandene Messung aus der simulierten Verteilung stammt. Als Beispiel wird der $p_0$ Wert genommen, der aus der $\chi^2$ berechnet wird die für ein Histogramm wie folgt definiert ist: $$ \chi^2 = \sum_{i=1}^N \frac{(n_i - y_i)^2}{\sigma_i^2} \, .$$ $\sigma_i$ ist die Unsicherheit des jeweiligen Bineintrags und entspricht $\sqrt{y_i}$. $y_i$ ist die Erwartete Anzahl an Ereignissen in einem Bineintrag ausgehend von der an die Messung skalierten Simulation des jeweiligen Bins. $n_i$ ist die Anzahl an gemessenen Ereignissen in dem jeweiligen Bin. 
Der Wert von $\chi^2$ entspricht dabei einer Abweichung der Messung von den simulierten Werten. Jeder der einzelnen Messwerte wird dabei entsprechend seiner Unsicherheit gewichtet. Messwerte mit großer Ungenauigkeit ändern den Gesamtwert weniger als Messwerte mit einer kleineren statistischen Unsicherheit.

Der so berechnete Wert kann in ein $p$ Wert übersetzt werden. Dieser ist ein Maß für die Bestätigung einer Hypothese und sagt aus, ob die Messung die Erwartung bestätigt. Nach Definition ist dieser eine Wahrscheinlichkeit die beobachtete Messung zu erhalten unter der Bedingung, dass die verwendete Hypothese stimmt. Sollte $p_0$ einen vorher festgelegten Wert unterschreiten (gewählt wird oft $0,05$ oder $0,01$) so kann die gewählte Hypothese - die Erwartungen $y_i$ zugunsten einer neuen Hypothese verworfen werden.

In [ ]:
import scipy.stats as scst

def p0_aus_chi2(messung, erwartung):
    # (mess, erw) ist ein Paar aus den Paaren (messung, erwartung)
    chi2_ = sum((1.0/erw) * (mess - erw) ** 2 for (mess, erw) in zip(messung, erwartung) if float(erw) != 0.0)
    # Übersetzung in die Wahrscheinlichkeit dass die Messung der Erwartung entspricht
    p0_ = 1.0 - scst.chi2.cdf(chi2_, df=len(messung) - 1)
    # Rückgabe: Name der Größe und die Größe selbst (für die Konvertierung)
    return r"$p_0$", p0_

In [ ]:
ww.own_statistical_evaluation_func = p0_aus_chi2
ww.run